In [1]:

from entity_retrieval import surface_index_memory
from itertools import product
from SPARQLWrapper import SPARQLWrapper, JSON
from tqdm import tqdm
import ujson
import re
import requests
from sklearn.preprocessing import normalize


In [2]:
from nor_to_sexpr import convert_s_expression_to_sparql

In [3]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\TOPU\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [4]:
surface_index = surface_index_memory.EntitySurfaceIndexMemory(
    "vi_entity_list_file_wikidata_complete_all_mention", "vi_entity_surface_map_file_wikidata_complete_all_mention","vi_wiki_complete_all_mention" )

INFO:entity_retrieval.surface_index_memory:Loading entity vocabulary from disk.
INFO:entity_retrieval.surface_index_memory:Loading surfaces from disk.
INFO:entity_retrieval.surface_index_memory:Done initializing surface index.


In [5]:
def is_valid_expression(expr):
    """Kiểm tra tính hợp lệ của biểu thức bằng cách đếm số ngoặc mở và đóng."""
    count = 0
    for char in expr:
        if char == '(':
            count += 1
        elif char == ')':
            count -= 1
        if count < 0:
            return False  # Gặp ngoặc đóng trước ngoặc mở
    return count == 0

def fix_unbalanced_parentheses(expr):
    """Loại bỏ ngoặc đóng dư nếu có."""
    while not is_valid_expression(expr) and expr.endswith(')'):
        expr = expr[:-1]
    return expr

In [6]:
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load model & tokenizer
tokenizer = AutoTokenizer.from_pretrained("VoVanPhuc/sup-SimCSE-VietNamese-phobert-base")
model = AutoModel.from_pretrained("VoVanPhuc/sup-SimCSE-VietNamese-phobert-base")


def get_embedding(text):
    """Trích xuất vector embedding từ mô hình"""
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)

    # Mean pooling
    embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
    return embeddings

def find_entity_or_relation(label, label_map, facc1_index, top_k=50, similarity_threshold=0.2):
    """
    Tìm thực thể hoặc quan hệ từ gold maps, SimCSE hoặc FACC1.
    """
    label_lower = label.lower()

    # Nếu có trong label_map, trả về ngay
    if label_map and label_lower in label_map:
        return label_map[label_lower]

    # Lấy embedding cho label
    label_embedding = get_embedding(label_lower).reshape(1, -1)

    if label_map:
        label_keys = list(label_map.keys())
        label_embeddings = np.array([get_embedding(k) for k in label_keys]).squeeze(1)

        # Đảm bảo đúng shape
        if len(label_embeddings.shape) == 1:
            label_embeddings = label_embeddings.reshape(1, -1)

        # Tính cosine similarity
        similarities = cosine_similarity(label_embedding, label_embeddings).flatten()

        # Chọn thực thể gần nhất
        merged_list = list(zip(label_keys, similarities))
        sorted_list = sorted(merged_list, key=lambda x: x[1], reverse=True)

        if sorted_list and sorted_list[0][1] > similarity_threshold:
            return label_map[sorted_list[0][0]]

    # Nếu không tìm thấy, thử trong KB (FACC1)
    facc1_cand_entities = facc1_index.get_indexrange_entity_el_pro_one_mention(label_lower, top_k=top_k)
    if facc1_cand_entities:
        best_match = max(facc1_cand_entities.items(), key=lambda x: x[1])
        return best_match[0]

    return label  # Trả về label nếu không tìm thấy

def parse_nsexpr(expr):
    """
    Chuyển chuỗi biểu thức thành cây cấu trúc dạng nested list.
    Hàm này dùng duyệt ký tự, khi gặp '(' sẽ tìm phần con cho đến khi khớp với ')',
    và giữ nguyên nội dung trong ngoặc vuông.
    """
    tokens = []
    i = 0
    while i < len(expr):
        if expr[i].isspace():
            i += 1
        elif expr[i] == '(':
            # Tìm phần con của biểu thức trong ngoặc đơn
            count = 1
            j = i + 1
            while j < len(expr) and count > 0:
                if expr[j] == '(':
                    count += 1
                elif expr[j] == ')':
                    count -= 1
                j += 1
            # Đệ quy phân tích phần con (loại bỏ ngoặc bao ngoài)
            subtree = parse_nsexpr(expr[i+1:j-1])
            tokens.append(subtree)
            i = j
        elif expr[i] == '[':
            # Giữ nguyên nội dung trong ngoặc vuông
            j = expr.find(']', i)
            if j == -1:
                return ""
                raise ValueError("Không tìm thấy dấu ']' kết thúc.")
                
            token = expr[i:j+1].strip()
            tokens.append(token)
            i = j + 1
        else:
            # Đọc một token cho đến khi gặp khoảng trắng hoặc ngoặc
            j = i
            while j < len(expr) and (not expr[j].isspace()) and expr[j] not in ['(', ')']:
                j += 1
            tokens.append(expr[i:j])
            i = j
    return tokens

def collect_labels(tree):
    """
    Duyệt cây cấu trúc (nested list) để thu thập các nhãn của quan hệ và thực thể.
    Giả sử:
      - Biểu thức JOIN có dạng: ["JOIN", relation_part, entity_part]
      - Phần relation_part: nếu là list và bắt đầu bằng "R", thì phần thứ hai chứa nhãn quan hệ (dạng "[ label ]"). 
        Nếu là chuỗi dạng "[ label ]" thì đó cũng là nhãn quan hệ.
      - Phần entity_part: nếu là chuỗi dạng "[ label ]" thì đó là nhãn thực thể, nếu là list thì xử lý đệ quy.
      - Biểu thức AND sẽ có nhiều biểu thức con.
    """
    relations = []
    entities = []
    
    if isinstance(tree, list) and tree:
        # Nếu token đầu tiên là JOIN hoặc AND
        op = tree[0]
        if isinstance(op, str):
            op_upper = op.upper()
        else:
            op_upper = ""
        
        if op_upper == "JOIN":
            # Xử lý phần quan hệ
            if len(tree) >= 2:
                rel_part = tree[1]
                # Nếu là list dạng [ "R", "[ label ]" ]
                if isinstance(rel_part, list) and len(rel_part) >= 2 and isinstance(rel_part[0], str) and rel_part[0].upper() == "R":
                    token = rel_part[1]
                    if isinstance(token, str) and token.startswith('[') and token.endswith(']'):
                        rel_label = token[1:-1].strip()
                        relations.append(rel_label)
                # Nếu là chuỗi dạng "[ label ]"
                elif isinstance(rel_part, str) and rel_part.startswith('[') and rel_part.endswith(']'):
                    rel_label = rel_part[1:-1].strip()
                    relations.append(rel_label)
                else:
                    # Nếu không đúng định dạng, duyệt đệ quy
                    sub_rel, sub_ent = collect_labels(rel_part)
                    relations.extend(sub_rel)
                    entities.extend(sub_ent)
            # Xử lý phần thực thể
            if len(tree) >= 3:
                ent_part = tree[2]
                if isinstance(ent_part, list):
                    sub_rel, sub_ent = collect_labels(ent_part)
                    relations.extend(sub_rel)
                    entities.extend(sub_ent)
                elif isinstance(ent_part, str) and ent_part.startswith('[') and ent_part.endswith(']'):
                    ent_label = ent_part[1:-1].strip()
                    entities.append(ent_label)
        elif op_upper == "AND":
            # Với AND, duyệt tất cả các phần con
            for sub in tree[1:]:
                sub_rel, sub_ent = collect_labels(sub)
                relations.extend(sub_rel)
                entities.extend(sub_ent)
        else:
            # Nếu không phải JOIN hay AND, duyệt tất cả các phần tử nếu chúng là list
            for elem in tree:
                if isinstance(elem, list):
                    sub_rel, sub_ent = collect_labels(elem)
                    relations.extend(sub_rel)
                    entities.extend(sub_ent)
    return relations, entities



def extract_entities_and_relations(normed_expr):

    if not normed_expr or len(normed_expr) == 0:  # Kiểm tra nếu normed_expr rỗng
        return [], []
    
    if normed_expr[0] != "(":
        return [], []
    
    tree = parse_nsexpr(normed_expr)
    if tree is None:
        return [], []  # Trả về danh sách rỗng nếu parse thất bại
    
    return collect_labels(tree)



c:\Users\TOPU\anaconda3\envs\py38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
def find_entity(label, label_map, facc1_index, top_k=20, similarity_threshold=0.2):
    """
    Tìm thực thể hoặc quan hệ từ gold maps, SimCSE hoặc FACC1.
    """
    label_lower = label.lower()

    # Nếu có trong label_map, trả về ngay
    if label_map and label_lower in label_map:
        return label_map[label_lower]

    # Lấy embedding cho label
    label_embedding = get_embedding(label_lower).reshape(1, -1)

    if label_map:
        label_keys = list(label_map.keys())
        label_embeddings = np.array([get_embedding(k) for k in label_keys]).squeeze(1)
        label_embedding = normalize(label_embedding, axis=1)
        label_embeddings = normalize(label_embeddings, axis=1)
        # Tính cosine similarity
        similarities = cosine_similarity(label_embedding, label_embeddings).flatten()
        # Chọn thực thể gần nhất
        merged_list = list(zip(label_keys, similarities))
        sorted_list = sorted(merged_list, key=lambda x: x[1], reverse=True)

        if sorted_list and sorted_list[0][1] > similarity_threshold:
            return label_map[sorted_list[0][0]]

    # Nếu không tìm thấy, thử trong KB (FACC1)
    facc1_cand_entities = facc1_index.get_indexrange_entity_el_pro_one_mention(label_lower, top_k=top_k)
    if facc1_cand_entities:
        temp = []
        for key in list(facc1_cand_entities.keys())[1:]:
            if facc1_cand_entities[key] >= 0.001:
                temp.append(key)
        if len(temp) > 0:
            label = [list(facc1_cand_entities.keys())[0]]+temp
        else:
            label = list(facc1_cand_entities.keys())[0]

    return label  # Trả về label nếu không tìm thấy

In [8]:
def find_relation(label, label_map, relation_kb_map, facc1_index, top_k=20, similarity_threshold=0.2):
    """
    Tìm quan hệ từ gold maps, SimCSE hoặc FACC1.
    """
    label_lower = label.lower()

    # 🔹 BƯỚC 1: Kiểm tra nếu đã có sẵn trong `label_map`
    if label_map and label_lower in label_map:
        return label_map[label_lower]

    # 🔹 BƯỚC 2: Kiểm tra nếu có sẵn trong `relation_kb_map`
    if relation_kb_map and label_lower in relation_kb_map:
        return relation_kb_map[label_lower]  # ⏩ Trả về ngay nếu có sẵn

    # 🔹 BƯỚC 3: Tính toán embedding và tìm quan hệ gần nhất bằng cosine similarity
    label_embedding = get_embedding(label_lower).reshape(1, -1)

    if label_map:
        label_keys = list(label_map.keys())
        label_embeddings = np.array([get_embedding(k) for k in label_keys]).squeeze(1)
        label_embedding = normalize(label_embedding, axis=1)
        label_embeddings = normalize(label_embeddings, axis=1)

        similarities = cosine_similarity(label_embedding, label_embeddings).flatten()
        sorted_list = sorted(zip(label_keys, similarities), key=lambda x: x[1], reverse=True)

        if sorted_list and sorted_list[0][1] > similarity_threshold:
            return label_map[sorted_list[0][0]]

    if relation_kb_map:
        label_keys = list(relation_kb_map.keys())
        label_embeddings = np.array([get_embedding(k) for k in label_keys]).squeeze(1)
        label_embedding = normalize(label_embedding, axis=1)
        label_embeddings = normalize(label_embeddings, axis=1)

        similarities = cosine_similarity(label_embedding, label_embeddings).flatten()
        sorted_list = sorted(zip(label_keys, similarities), key=lambda x: x[1], reverse=True)

        if sorted_list and sorted_list[0][1] > similarity_threshold:
            return relation_kb_map[sorted_list[0][0]]

    return label

In [9]:
class SExpressionParser:
    def __init__(self):
        self.var_counter = 1  # Đếm số biến trung gian (?X1, ?X2, ...)

    def get_new_var(self):
        """Tạo biến trung gian mới."""
        var_name = f"?X{self.var_counter}"
        self.var_counter += 1
        return var_name

    def parse_s_expr(self, s_expr):
        """Chuyển đổi S-Expression thành danh sách lồng nhau."""
        s_expr = re.sub(r'\(', ' ( ', s_expr)
        s_expr = re.sub(r'\)', ' ) ', s_expr)
        tokens = s_expr.split()
        return self.build_tree(tokens)

    def build_tree(self, tokens):
        """Chuyển đổi danh sách token thành cây lồng nhau."""
        if not tokens:
            return None
        token = tokens.pop(0)
        if token == "(":
            sub_expr = []
            while tokens[0] != ")":
                sub_expr.append(self.build_tree(tokens))
            tokens.pop(0)  # Bỏ dấu ")"
            return sub_expr
        elif token == ")":
            raise ValueError("Unexpected ')'")
        else:
            return token

    def process_join(self, expr, target_var):
        """
        Xử lý JOIN, tạo triple SPARQL.
        """
        triples = []
        if not isinstance(expr, list):
            return expr, triples

        if expr[0] == "AND":
            # Xử lý từng JOIN trong AND riêng lẻ
            for sub_expr in expr[1:]:
                _, sub_triples = self.process_join(sub_expr, target_var)
                triples.extend(sub_triples)
            return target_var, triples

        if expr[0] == "JOIN":
            right_expr = expr[2]
            right_triples = []
            if isinstance(right_expr, list) and right_expr[0] == "JOIN":
                right_var, right_triples = self.process_join(right_expr, self.get_new_var())
            else: 
                right_var = right_expr
            
            # Xử lý nhánh trái
            left_expr = expr[1]
            if isinstance(left_expr, list) and left_expr[0] == "R":
                rel = left_expr[1]
                right = right_var
                left = target_var
                if right[0] != '?':
                    right = "wd:" + right
                if left[0] !='?':
                    left = "wd:" + left   
                triples.append([right, f"wdt:{rel}", left])
            else:
                right = right_var
                left = target_var
                if right[0] != '?':
                    right = "wd:" + right
                if left[0] !='?':
                    left = "wd:" + left   
                triples.append([left, f"wdt:{left_expr}", right])

            # Thêm các triples từ nhánh phải trước khi thêm triple chính
            triples = right_triples + triples
            return target_var, triples

        return expr, triples

    def s_expr_to_sparql(self, s_expr):
        """Chuyển đổi từ S-Expression sang SPARQL."""
        if s_expr.count("(") != s_expr.count(")"):
            return None
        if s_expr.count("[") != s_expr.count("]"):
            return None
        parsed_expr = self.parse_s_expr(s_expr)
        target_var = "?answer"
        final_var, triples = self.process_join(parsed_expr, target_var)

        sparql_body = "\n  ".join([" ".join(t) + " ." for t in triples])
        sparql_query = f"""PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
SELECT DISTINCT {target_var} WHERE {{ 
  {sparql_body}
}}"""
        return sparql_query

import requests    
WIKIDATA_SPARQL_ENDPOINT = "https://query.wikidata.org/sparql"
WIKIDATA_API_ENDPOINT = "https://www.wikidata.org/w/api.php"

def execute_query_with_odbc(sparql_query):
    """Truy vấn Wikidata và trả về danh sách câu trả lời (bao gồm tất cả biến)"""
    headers = {"User-Agent": "Mozilla/5.0", "Accept": "application/sparql-results+json"}
    response = requests.get(WIKIDATA_SPARQL_ENDPOINT, params={"query": sparql_query, "format": "json"}, headers=headers)

    if response.status_code == 200:
        results = response.json().get("results", {}).get("bindings", [])
        answers = []

        for result in results:
            for var in result:  # Duyệt qua tất cả các biến trả về
                value = result[var]["value"]
                answers.append(value)  # Chấp nhận tất cả giá trị, không chỉ thực thể Wikidata

        return answers  # Trả về toàn bộ danh sách kết quả

    return []


In [10]:
def convert_normed_to_s_expression(normed_expr, gold_relation_map, gold_entity_map, relation_KB_map, facc1_index):
    """
    Chuyển đổi từ normed_sexpression sang s_expression.
    Sau khi trích xuất các nhãn quan hệ và thực thể từ normed_expr,
    ta lấy danh sách các mã ứng viên cho mỗi nhãn và tạo hoán vị giữa các cặp ứng viên đó.
    Kết quả trả về là một danh sách các s_expression khả dĩ.
    """
    normed_expr = fix_unbalanced_parentheses(normed_expr)
    # Trích xuất các nhãn quan hệ và thực thể từ biểu thức
    relations, entities = extract_entities_and_relations(normed_expr)
    
    # Tạo mapping từ token xuất hiện trong biểu thức sang danh sách các ứng viên mã.
    # Ví dụ: token_str = "[ author ]"
    candidate_map = {}
    for rel in relations:
        token = f'[ {rel} ]'
        candidate = find_relation(rel, gold_relation_map, relation_KB_map, facc1_index)
        # Nếu candidate không phải danh sách, chuyển nó thành danh sách để tạo hoán vị
        if not isinstance(candidate, list):
            candidate = [candidate]
        candidate_map[token] = candidate
        
    for ent in entities:
        token = f'[ {ent} ]'
        candidate = find_entity(ent, gold_entity_map , facc1_index)
        if not isinstance(candidate, list):
            candidate = [candidate]
        candidate_map[token] = candidate
    
    # Nếu không có token nào cần thay thế, trả về biểu thức gốc
    if not candidate_map:
        return [normed_expr]
    
    # Lấy danh sách các token và danh sách các danh sách ứng viên tương ứng
    tokens = list(candidate_map.keys())
    candidate_lists = [candidate_map[token] for token in tokens]
    
    # Tạo tất cả các hoán vị ứng viên (Cartesian product)
    all_combinations = list(product(*candidate_lists))
    
    s_expressions = []
    for comb in all_combinations:
        temp_expr = normed_expr
        # Với mỗi token, thay thế bằng ứng viên tương ứng theo hoán vị
        for token, replacement in zip(tokens, comb):
            temp_expr = temp_expr.replace(token, replacement)
        s_expressions.append(temp_expr)
    
    return s_expressions

In [11]:

def load_jsonl(file_path):
    """Đọc file JSONL và trả về danh sách các object"""


    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read().replace('(EXPECTED RESULT)', 'null').replace('(QUESTION)', 'null')

    try:
        data = ujson.loads(content)
        print("JSON loaded successfully!")
    except Exception as e:
        print(f"Error: {e}")
    return data


In [12]:
def calculate_prf1(gold_answers, pred_answers):
    """Tính Precision, Recall, F1-score"""
    if len(gold_answers) == 0:
        if len(pred_answers) == 0:
            return [1.0, 1.0, 1.0]  # Đúng khi không có câu trả lời
        else:
            return [0.0, 1.0, 0.0]
    elif len(pred_answers) == 0:
        return [0.0, 0.0, 0.0]
    tp = 1e-40  # numerical trick

    tp = tp + len(set(gold_answers) & set(pred_answers))

    fp = len(set(pred_answers) - set(gold_answers))
    fn = len(set(gold_answers) - set(pred_answers))
    precision = tp / (tp + fp) 
    recall = tp / (tp + fn) 
    f1 = (2 * precision * recall) / (precision + recall) 

    return [precision, recall, f1]


In [13]:
# Đọc dữ liệu
predictions = load_jsonl("LLMs/beam_prediction/generated_predictions_beam_Q7b.json")
gold_data = load_jsonl("Data/LC-QuAD2.0/label_map/LC-QuAD2.0_test.json")
gold_data = gold_data[:1000]

JSON loaded successfully!
JSON loaded successfully!


### Load danh sách quan hệ trong KB

In [14]:
file_path = "property_list_file_wikidata_complete_all_mention"

relation_KB_map = {}

# Đọc file và đảo ngược quan hệ
with open(file_path, "r", encoding="utf-8") as f:
    for line in f:
        parts = line.strip().split("\t")  # Tách theo tab
        if len(parts) == 2:
            relation_KB_map[parts[1]] = parts[0]

In [15]:
ex_cnt = 0
top_hit = 0
failed_preds = []
final_executable_cnt = 0

In [16]:
import json
import time
from tqdm import tqdm

BATCH_SIZE = 300  # Số dòng mỗi batch
LOG_FILE = "progress_log.txt"

results = []
simcse_model = model  # Mô hình SimCSE của bạn
facc1_index = surface_index
parser = SExpressionParser()
start_time = time.perf_counter()  # Bắt đầu đo thời gian

# Ghi log
with open(LOG_FILE, "w", encoding="utf-8") as log_file:
    log_file.write("Bắt đầu quá trình đánh giá\n")

for i, (pred, gold) in enumerate(tqdm(zip(predictions, gold_data), total=len(predictions), desc="Đang đánh giá")):
    try:
        gold_entity_map = {v.lower(): k for k, v in gold['gold_entity_map'].items()}
        gold_relation_map = {v.lower(): k for k, v in gold['gold_relation_map'].items()}
        gold_answers = gold.get("answer", [])
        executable_index = None
        best_f1, best_precision, best_recall = 0, 0, 0
        kq = []
        lag_result = False
        denormed_pred = []
        for rank, query in enumerate(pred['predicted_query']):
            if lag_result:
                break

            set_query = convert_normed_to_s_expression(query, gold_relation_map, gold_entity_map, relation_KB_map , facc1_index)
            for q in set_query:
                query_result = []
                if not q:
                    continue
                if rank == 0 and q.lower() ==gold['s_expr'].lower():
                    ex_cnt +=1
                sparql = convert_s_expression_to_sparql(q)
                if sparql == "UNKNOWN":
                    continue
                denormed_pred.append(sparql)
                query_result = execute_query_with_odbc(sparql)
                if query_result:
                    if rank == 0:
                        top_hit += 1
                    executable_index = rank
                    precision, recall, f1 = calculate_prf1(gold_answers, query_result)
                    if f1 > best_f1:
                        kq = query_result
                        best_f1, best_precision, best_recall = f1, precision, recall
                    if precision == 1:
                        lag_result = True
                        break
        if executable_index is not None:
            final_executable_cnt+=1
        else:
            failed_preds.append({'qid':gold["question_id"], 
                'gt_sexpr': gold['s_expr'], 
                'gt_normed_sexpr': pred['gen_label'],
                'pred': pred, 
                'denormed_pred':denormed_pred})    
        results.append({
            "qid": gold["question_id"],
            "answer": gold_answers,
            "result": kq,
            "nor_s_expr":gold["nor_s_expr"],
            "precision": best_precision,
            "recall": best_recall,
            "f1": best_f1
        })

        # Ghi log tiến trình xử lý
        with open(LOG_FILE, "a", encoding="utf-8") as log_file:
            log_file.write(f"Đã xử lý xong dòng {i + 1}/{len(predictions)}\n")

        # Khi đủ 100 kết quả, lưu vào file và reset biến `results`
        if (i + 1) % BATCH_SIZE == 0 or (i + 1) == len(predictions):
            batch_id = (i + 1) // BATCH_SIZE
            filename = f"LLMs/eval_result/evaluation_vinallamaQ7b_part_{batch_id}.json"
            with open(filename, "w", encoding="utf-8") as f:
                json.dump(results, f, indent=4)
            print(f"✅ Đã lưu {len(results)} dòng vào {filename}")
            results = []  # Reset danh sách kết quả

    except Exception as e:
        with open(LOG_FILE, "a", encoding="utf-8") as log_file:
            log_file.write(f"Lỗi tại dòng {i + 1}: {str(e)}\n")
        print(f"❌ Lỗi tại dòng {i + 1}: {e}")

# Kết thúc đo thời gian
end_time = time.perf_counter()
total_time = end_time - start_time
print(f"🎯 Quá trình đánh giá hoàn tất trong {total_time:.2f} giây!")

with open(LOG_FILE, "a", encoding="utf-8") as log_file:
    log_file.write(f"Quá trình đánh giá hoàn tất trong {total_time:.2f} giây!\n")


Đang đánh giá:   0%|          | 0/1000 [05:12<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
print('STR Match', ex_cnt/ len(predictions))
print('TOP 1 Executable', top_hit/ len(predictions))
print('Final Executable', final_executable_cnt/ len(predictions))

STR Match 0.34690943938667945
TOP 1 Executable 0.45759463344513657
Final Executable 0.6411116435074269


In [ ]:
STR_Match = ex_cnt/ len(predictions)
TOP1_Executable = top_hit/ len(predictions)
Final_Executable = final_executable_cnt/ len(predictions)

In [ ]:
import glob

# Danh sách các file kết quả đánh giá
file_list = glob.glob("LLMs/eval_result/evaluation_vinallamaQ7b_part_*.json")

# Biến để tổng hợp kết quả
total_precision = 0
total_recall = 0
total_f1 = 0
total_samples = 0

# Đọc từng file và tổng hợp dữ liệu
for file in file_list:
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)
        for entry in data:
            total_precision += entry.get("precision", 0)
            total_recall += entry.get("recall", 0)
            total_f1 += entry.get("f1", 0)
            total_samples += 1

# Tránh chia cho 0
if total_samples > 0:
    avg_precision = total_precision / total_samples
    avg_recall = total_recall / total_samples
    avg_f1 = total_f1 / total_samples
else:
    avg_precision, avg_recall, avg_f1, hits_at_1 = 0, 0, 0, 0

# Hiển thị kết quả
print(f"📊 Precision: {avg_precision:.5f}")
print(f"📊 Recall: {avg_recall:.5f}")
print(f"📊 F1-score: {avg_f1:.5f}")


📊 Precision: 0.51899
📊 Recall: 0.53568
📊 F1-score: 0.51993


In [ ]:
# Tạo dữ liệu kết quả đánh giá
eval_results = {
    "precision": avg_precision,
    "recall": avg_recall,
    "f1": avg_f1,
    "STR Match": STR_Match,
    "Hit@1": TOP1_Executable,
    "Final_Executable": Final_Executable, 
}

# Lưu vào file JSON
output_file = "LLMs/eval_result/Final_evaluation_Q7b.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(eval_results, f, indent=2, ensure_ascii=False)

print(f"✅ Kết quả đã được lưu vào {output_file}")

✅ Kết quả đã được lưu vào LLMs/eval_result/Final_evaluation_27b.json


In [ ]:

output_file = "LLMs/eval_result/Failed_result_Q7b.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(failed_preds, f, indent=2, ensure_ascii=False)

print(f"✅ Kết quả đã được lưu vào {output_file}")

✅ Kết quả đã được lưu vào LLMs/eval_result/Failed_result_27b.json
